In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from IPython.display import Audio

# Some plotting niceties
plt.rc('figure', figsize=(10, 4))
sns.set_style('white')
sns.set_style('ticks')

import nengo
import phd

def img(array):
    plt.pcolormesh(array.T)
    plt.ylim(top=array.shape[1])
    plt.xlim(right=array.shape[0])
    plt.yticks(())
    sns.despine(left=True)
    plt.tight_layout()

In [ ]:
%%javascript
if($(IPython.toolbar.selector.concat(' > #kill-run-first')).length == 0){
  IPython.toolbar.add_buttons_group([
    {
      'label'   : 'kill and run-first',
      'icon'    : 'fa fa-angle-double-down',
      'callback': function(){
        IPython.notebook.kernel.restart();
        $(IPython.events).one('kernel_ready.Kernel', function(){
          var idx = IPython.notebook.get_selected_index();
          IPython.notebook.select(0);
          IPython.notebook.execute_cell();
          IPython.notebook.select(idx);
        });
      }
    }
  ], 'kill-run-first');
}

## Basic usage

In [ ]:
model = phd.sermo.Production()
model.trial.sequence = 'BLAA*POS1 + TII*POS2 + DAS*POS3'
freqs = [('ges-de-ccv', 'blaa', 1.2), ('ges-de-cv', 'tii', 1.8), ('ges-de-cvc', 'das', 1.1)]
t = 0.2
for gdir, ges, freq in freqs:
    path = phd.ges_path(gdir, '%s.ges' % ges.lower())
    traj = phd.vtl.parse_ges(path).trajectory(dt=model.trial.dt)
    model.add_syllable(label=ges.upper(), freq=freq, trajectory=traj)
    t += 1. / freq

In [ ]:
net = model.build()
with net:
    p_syll = nengo.Probe(net.sequence.syllable.output, synapse=0.01)
    p_gate = nengo.Probe(net.sequencer.gate, synapse=0.01)
    p_reset = nengo.Probe(net.sequencer.reset, synapse=0.01)
    p_timer = nengo.Probe(net.sequencer.timer, synapse=0.01)
    p_disinhib = [nengo.Probe(dmp.disinhibit, synapse=0.01) for dmp in net.syllables]
    p_dmps = [nengo.Probe(dmp.osc, synapse=0.01) for dmp in net.syllables]
    p_prod = nengo.Probe(net.production_info.output, synapse=0.01)

In [ ]:
sim = nengo.Simulator(net)
sim.run(t)

In [ ]:
syllables = [s[1].upper() for s in freqs]
plt.plot(sim.trange(), 
         nengo.spa.similarity(sim.data[p_syll],
                              net.sequence.vocab.create_subset(syllables)))
plt.legend(syllables, loc='best')

In [ ]:
plt.plot(sim.trange(), sim.data[p_gate])
plt.plot(sim.trange(), sim.data[p_reset])
plt.plot(sim.trange(), sim.data[p_timer])
plt.legend(['gate', 'kick', 'timer', 'timer'], loc="best")

In [ ]:
for p_dis in p_disinhib:
    plt.plot(sim.trange(), sim.data[p_dis])
plt.legend(syllables, loc="best")

In [ ]:
plt.figure(figsize=(10, 10))
plt.subplot(2, 2, 1)
plt.plot(sim.data[p_timer].T[0], sim.data[p_timer].T[1])
for i, p_dmp in enumerate(p_dmps):
    plt.subplot(2, 2, i + 2)
    plt.plot(sim.data[p_dmp].T[0], sim.data[p_dmp].T[1])

In [ ]:
plt.plot(sim.trange(), sim.data[p_prod]);

## Running an experiment

In [ ]:
model = phd.sermo.Production()
syllables = [('ges-de-ccv', 'blaa', 1.2), ('ges-de-cv', 'tii', 1.8), ('ges-de-cvc', 'das', 1.1)]
sequence = ['blaa' , 'tii', 'das']
expt = phd.experiments.ProductionExperiment(model, syllables, sequence)
res = expt.run()

In [ ]:
img(res.traj)

In [ ]:
img(res.simtraj)
print(res.simrmse)

In [ ]:
img(res.reconstructed)
print(res.reconstructedrmse)

In [ ]:
print(res.audio.shape[0] / float(res.fs))
print(res.traj.shape)
Audio(data=res.audio.flatten(), rate=res.fs)

In [ ]:
targets = [phd.vtl.parse_ges(phd.ges_path(gdir, "%s.ges" % ges.lower()))
           for gdir, ges, freq in expt.syllables]
plt.subplot(1, 3, 1)
img(targets[0].trajectory(0.001))
plt.subplot(1, 3, 2)
img(targets[1].trajectory(0.001))
plt.subplot(1, 3, 3)
img(targets[2].trajectory(0.001))

In [ ]:
from phd import analysis
reload(analysis)
comb = phd.analysis.gs_combine(targets)
img(comb.trajectory(0.001))

In [ ]:
reload(analysis)
astring = phd.analysis.gs2strings(comb)
astring

In [ ]:
gs = phd.experiments.gesture_score(res.simtraj, model.trial.dt)
gstring = phd.analysis.gs2strings(gs)
gstring

In [ ]:
import nwalign as nw
left, right = nw.global_align(astring, gstring)
print left
print right
n_g = len(astring)
n_sub = n_del = n_ins = 0
# Note! right is the reconstructed!
for lc, rc in zip(left, right):
    if lc == '-':
        n_ins += 1
    elif rc == '-':
        n_del += 1
    elif lc != rc:
        n_sub += 1

acc = float(n_g - n_sub - n_del - n_ins) / n_g
acc